In [21]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy.stats import boxcox
from precision_recall_cutoff import precision_recall_cutoff
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE, RFECV

from sklearn.metrics import recall_score, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'turnover.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
turnover = pd.read_csv(file_content_stream)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [22]:
## changing 'sales' and 'salary' to dummy variables
turnover = pd.concat([turnover.drop(columns = ['sales', 'salary'], axis = 1), pd.get_dummies(turnover[['sales', 'salary']])], axis = 1)
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0


In [23]:
## Engineering interactions
turnover['interaction_1'] = turnover['satisfaction_level'] * turnover['time_spend_company']
turnover['interaction_2'] = turnover['last_evaluation'] * turnover['promotion_last_5years']
turnover.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium,interaction_1,interaction_2
0,0.38,0.53,2,157,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1.14,0.0
1,0.80,0.86,5,262,6,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,4.80,0.0
2,0.11,0.88,7,272,4,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0.44,0.0
3,0.72,0.87,5,223,5,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,3.60,0.0
4,0.37,0.52,2,159,3,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1.11,0.0


In [24]:
## defining input and target
x = turnover.drop(columns = 'left', axis = 1)
y = turnover['left']

## splitting the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [25]:
## defining the scaler
scaler = MinMaxScaler()

x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:484: RuntimeWarning: divide by zero encountered in true_divide
  self.scale_ = (feature_range[1] - feature_range[0]) / _handle_zeros_in_scale(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:484: RuntimeWarning: divide by zero encountered in true_divide
  self.scale_ = (feature_range[1] - feature_range[0]) / _handle_zeros_in_scale(


## RFECV

#### Logistic Regression

In [15]:
logit_rfe = RFECV(estimator = LogisticRegression(), min_features_to_select = 2, cv = 3).fit(x_train, y_train)

## identifying features to be selected
print(x_train.columns[logit_rfe.support_])

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_IT', 'sales_RandD', 'sales_hr',
       'sales_management', 'sales_product_mng', 'sales_technical',
       'salary_high', 'salary_low', 'salary_medium', 'interaction_1',
       'interaction_2'],
      dtype='object')


In [16]:
## defining inputs and targets
x_train_1 = x_train[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_IT', 'sales_RandD', 'sales_hr',
       'sales_management', 'sales_product_mng', 'sales_technical',
       'salary_high', 'salary_low', 'salary_medium', 'interaction_1',
       'interaction_2']]

x_test_1 = x_test[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales_IT', 'sales_RandD', 'sales_hr',
       'sales_management', 'sales_product_mng', 'sales_technical',
       'salary_high', 'salary_low', 'salary_medium', 'interaction_1',
       'interaction_2']]

## building logistic regression model
logit_md = LogisticRegression().fit(x_train_1, y_train)

## predicting on the testing
logit_preds = logit_md.predict_proba(x_test_1)[:, 1]

## changing likelihoods to labes
logit_labels = precision_recall_cutoff(y_test, logit_preds)

## printing classification report
print(classification_report(y_test, logit_labels))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86      2286
           1       0.55      0.71      0.62       714

    accuracy                           0.80      3000
   macro avg       0.73      0.77      0.74      3000
weighted avg       0.82      0.80      0.80      3000



#### Random Forest

In [17]:
rf_rfe = RFECV(estimator = RandomForestClassifier(n_estimators = 500, max_depth = 3), 
               step = 1, min_features_to_select = 2, cv = 3).fit(x_train, y_train)

print(x_train.columns[rf_rfe.support_])

Index(['satisfaction_level', 'number_project', 'time_spend_company',
       'interaction_1'],
      dtype='object')


In [27]:
x_train_2 = x_train[['satisfaction_level', 'number_project', 'time_spend_company', 'interaction_1']]

x_test_2 = x_test[['satisfaction_level', 'number_project', 'time_spend_company','interaction_1']]
    
## building logistic regression model
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(x_train_2, y_train)

## predicting on the testing
rf_preds = rf_md.predict_proba(x_test_2)[:, 1]

## changing likelihoods to labes
rf_labels = precision_recall_cutoff(y_test, rf_preds)

## printing classification report
print(classification_report(y_test, rf_labels))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      2286
           1       0.92      0.90      0.91       714

    accuracy                           0.96      3000
   macro avg       0.94      0.94      0.94      3000
weighted avg       0.96      0.96      0.96      3000



Based on the above classification reports, I would choose the Random Forest model to predict 'left' because the Random Forest had a higher performance.